In [39]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [40]:
pip install requests

In [87]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/dc/72/96b5afa16908f9abc7c24b70adfd3a46c9740eb728ddfeab28379e38eaf9/selenium-4.16.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/14/fb/9299cf74953f473a15accfdbe2c15218e766bae8c796f2567c83bae03e98/trio-0.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for outcome from https://files.pythonhosted.org/packages/55/8b/5ab7257531a5d830fc8000c476e63c935488d74609b50f9384a643ec0a62/outcome-1.3.0.post0-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
     ---------------------------------------- 58.3/58.3 kB 1.5 MB/s eta 0:00:00
   ---------------

## Scrape Buying Data - Dublin

In [29]:
import requests
from bs4 import BeautifulSoup
import csv
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


url = "https://www.daft.ie/property-for-sale/dublin-city"

# Mimicing a browser to avoid getting 403 error
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
pg_iter = np.arange(0,2160, 20)

with open('Dublin-extract-buy2.csv', 'w', encoding='utf-8', newline='') as f: 
    write_to_csv=csv.writer(f)
    
    # iterating through all the pages
    for page in pg_iter:
        s = url+"?from="+str(page)+"&pageSize=0"
        #Getting response from daft
        response = requests.get(s, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            print("ok")

            soup = BeautifulSoup(response.content, 'html.parser')
            each_ad = soup.find_all('li', class_="SearchPagestyled__Result-v8jvjf-2 iWPGnb")
            lc=0
            pages = soup.find('p',attrs = {'data-testid':'pagination-results', 'class': 'SearchPagePagination__PaginationResults-sc-1gbvxa5-1 gzDAMl'})
            print(pages, type(pages))
            for l in each_ad:
                address=''
                price=''
                bed=''
                bath=''
                area=''
                property_type=''
                ag=''
                agent=''
                lc=lc+1
              #  print(lc)
                try:
                    try:
                        address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 dzihyY").text             #address 

                    except:    
                       # print("Different format : need to scrape by clicking link")

                        try:
                            address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 hCMmam").text             #address 

                        except AttributeError: 
                            print("we have an error with address")
                            continue                  

                    # Find all <a> tags within the current <li> (l)
                    all_links = l.find_all('a')

                   #print(lc)
                   # print(len(all_links))   

                    # Extract href attributes and print them

                    if len(all_links)==1:
                        for link in all_links:
                            new_url = "https://www.daft.ie/"+link.get('href')
                           # print(new_url)
                            soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')

                            try:                    
                                price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                            except:
                                print("error in price")

                            try:                    
                                bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                            except:
                                print("error in bed")

                            try:                    
                                bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                            except:
                                print("error in bath")

                            try:                    
                                area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                            except:
                                print("error in area")

                            try:                    
                                property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                            except:
                                print("error in type")


                    if len(all_links)>1:
                        a=0
                        for link in all_links:
                            if a>0:
                                new_url = "https://www.daft.ie/"+link.get('href')
                                #print(new_url)
                                soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')
                                try:                    
                                    price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                                except:
                                    print("error in price")

                                try:                    
                                    bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                                except:
                                    print("error in bed")
                                try:                    
                                    bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                                except:
                                    print("error in bath")

                                try:                    
                                    area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                                except:
                                    print("error in area")

                                try:                    
                                    property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                                except:
                                    print("error in type")

                            a=a+1

                    write_to_csv.writerow(["Dublin",new_url, address, price, bed, bath, area, property_type])


                except AttributeError:
                    print("not sure why the error")
                
                

ok
None <class 'NoneType'>
error in bath
error in area
error in bath
error in area
error in area
ok
None <class 'NoneType'>
error in area
ok
None <class 'NoneType'>
error in area
error in area
error in bath
error in bath
error in area
error in area
error in area
error in area
error in area
error in area
ok
None <class 'NoneType'>
error in bed
ok
None <class 'NoneType'>
error in area
error in area
error in area
error in area
ok
None <class 'NoneType'>
error in area
error in area
ok
None <class 'NoneType'>
error in area
error in area
error in area
error in area
error in area
error in bed
error in bath
error in bath
error in bath
error in bath
error in bath
ok
None <class 'NoneType'>
error in area
error in area
error in bath
error in area
error in area
ok
None <class 'NoneType'>
error in area
ok
None <class 'NoneType'>
error in area
ok
None <class 'NoneType'>
error in area
error in area
error in area
error in area
ok
None <class 'NoneType'>
error in area
error in area
error in area
ok
Non

error in bed
error in bath
ok
None <class 'NoneType'>
error in area
error in area
error in area
ok
None <class 'NoneType'>
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
ok
None <class 'NoneType'>
error in bed
error in bath
error in area
ok
None <class 'NoneType'>
error in area
error in area
ok
None <class 'NoneType'>
error in area
ok
None <class 'NoneType'>
error in bath
error in area
error in area
error in bed
error in bath
error in area
error in bed
error in bath
ok
None <class 'NoneType'>
error in bed
error in bath
error in bed
error in bath
error in area
error in area
ok
None <class 'NoneType'>
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in area
error in area
ok
None <class 'NoneType'>
error in area
error in area
error in bed
error in bath
error in area
error in area
ok
None <class 'NoneType'>
error in area
error in bed
error in bath
error in area
error in area
ok
None <class 'NoneType'>
er

In [8]:
import numpy as np

In [9]:
pg_iter = np.arange(0,2500, 20)

## Scrape Renting Data - Dublin

In [30]:
import requests
from bs4 import BeautifulSoup
import csv
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


url = "https://www.daft.ie/property-for-rent/dublin-city"

# Mimicing a browser to avoid getting 403 error
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
pg_iter = np.arange(0,1080, 20)

with open('Dublin-extract-rent2.csv', 'w', encoding='utf-8', newline='') as f: 
    write_to_csv=csv.writer(f)
    
    # iterating through all the pages
    for page in pg_iter:
        s = url+"?from="+str(page)+"&pageSize=0"
        #Getting response from daft
        response = requests.get(s, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            print("ok")

            soup = BeautifulSoup(response.content, 'html.parser')
            each_ad = soup.find_all('li', class_="SearchPagestyled__Result-v8jvjf-2 iWPGnb")
            lc=0
            
           # pages = soup.find('p',attrs = {'data-testid':'pagination-results', 'class': 'SearchPagePagination__PaginationResults-sc-1gbvxa5-1 gzDAMl'})
           # print(pages, type(pages))
            
            for l in each_ad:
                address=''
                price=''
                bed=''
                bath=''
                area=''
                property_type=''
                ag=''
                agent=''
                lc=lc+1
              #  print(lc)
                
                try:
                    try:
                        address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 dzihyY").text             #address 

                    except:    
                       # print("Different format : need to scrape by clicking link")

                        try:
                            address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 hCMmam").text             #address 

                        except AttributeError: 
                            print("we have an error with address")
                            continue                  

                    # Find all a tags within the current li
                    all_links = l.find_all('a')

                   #print(lc)
                   # print(len(all_links))   

                    # Extract href attributes and print them

                    if len(all_links)==1:
                        for link in all_links:
                            new_url = "https://www.daft.ie/"+link.get('href')
                           # print(new_url)
                            soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')

                            try:                    
                                price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                            except:
                                print("error in price")

                            try:                    
                                bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                            except:
                                print("error in bed")

                            try:                    
                                bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                            except:
                                print("error in bath")

                            try:                    
                                area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                            except:
                                print("error in area")

                            try:                    
                                property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                            except:
                                print("error in type")


                    if len(all_links)>1:
                        a=0
                        for link in all_links:
                            if a>0:
                                new_url = "https://www.daft.ie/"+link.get('href')
                                #print(new_url)
                                soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')
                                try:                    
                                    price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                                except:
                                    print("error in price")

                                try:                    
                                    bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                                except:
                                    print("error in bed")
                                try:                    
                                    bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                                except:
                                    print("error in bath")

                                try:                    
                                    area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                                except:
                                    print("error in area")

                                try:                    
                                    property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                                except:
                                    print("error in type")

                            a=a+1

                    write_to_csv.writerow(["Dublin",new_url, address, price, bed, bath, area, property_type])


                except AttributeError:
                    print("not sure why the error")
                
                

ok
error in area
error in area
error in area
error in area
error in area
error in bed
error in bath
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in area
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in bed
error in bath
error in area
error in area
error in area
ok
error in bed
error in bath
error in area
error in area
error in area
error in bed
error in bath
error in area
error in 

error in area
error in area
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in area
error in area
error in bed
error in bath
error in area
error in area
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
ok
error in bed
error in bath
error in area
error in area
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in bed
error in bath
error in area
error in area
error in area
error in bed
error in bath
error in area
error i

error in area
error in area
error in bed
error in bath
error in area
error in area
error in bed
error in bath
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
ok
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in bed
error in bath
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
ok
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
ok
error in area
error in area
error in area
error in area
error in area
error in area
error in bed
error in bath
error in area
error in area
error in area
error in area
e

## Scraping buying data - cork

In [31]:
import requests
from bs4 import BeautifulSoup
import csv
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


url = "https://www.daft.ie/property-for-sale/ireland"

# Mimicing a browser to avoid getting 403 error
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
pg_iter = np.arange(0,1700, 20)

with open('Cork-extract-buy2.csv', 'w', encoding='utf-8', newline='') as f: 
    write_to_csv=csv.writer(f)
    
    # iterating through all the pages
    for page in pg_iter:
        s = url+"?from="+str(page)+"&location=cork&location=cork-city&location=cork-commuter-towns-cork"
        #Getting response from daft
        response = requests.get(s, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            print("ok")

            soup = BeautifulSoup(response.content, 'html.parser')
            each_ad = soup.find_all('li', class_="SearchPagestyled__Result-v8jvjf-2 iWPGnb")
            lc=0
            
           # pages = soup.find('p',attrs = {'data-testid':'pagination-results', 'class': 'SearchPagePagination__PaginationResults-sc-1gbvxa5-1 gzDAMl'})
           # print(pages, type(pages))
            
            for l in each_ad:
                address=''
                price=''
                bed=''
                bath=''
                area=''
                property_type=''
                ag=''
                agent=''
                lc=lc+1
              #  print(lc)
                
                try:
                    try:
                        address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 dzihyY").text             #address 

                    except:    
                       # print("Different format : need to scrape by clicking link")

                        try:
                            address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 hCMmam").text             #address 

                        except AttributeError: 
                            print("we have an error with address")
                            continue                  

                    # Find all a tags within the current li
                    all_links = l.find_all('a')

                   #print(lc)
                   # print(len(all_links))   

                    # Extract href attributes and print them

                    if len(all_links)==1:
                        for link in all_links:
                            new_url = "https://www.daft.ie/"+link.get('href')
                           # print(new_url)
                            soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')

                            try:                    
                                price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                            except:
                                print("error in price")

                            try:                    
                                bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                            except:
                                print("error in bed")

                            try:                    
                                bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                            except:
                                print("error in bath")

                            try:                    
                                area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                            except:
                                print("error in area")

                            try:                    
                                property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                            except:
                                print("error in type")


                    if len(all_links)>1:
                        a=0
                        for link in all_links:
                            if a>0:
                                new_url = "https://www.daft.ie/"+link.get('href')
                                #print(new_url)
                                soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')
                                try:                    
                                    price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                                except:
                                    print("error in price")

                                try:                    
                                    bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                                except:
                                    print("error in bed")
                                try:                    
                                    bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                                except:
                                    print("error in bath")

                                try:                    
                                    area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                                except:
                                    print("error in area")

                                try:                    
                                    property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                                except:
                                    print("error in type")

                            a=a+1

                    write_to_csv.writerow(["Cork",new_url, address, price, bed, bath, area, property_type])


                except AttributeError:
                    print("not sure why the error")
                
                

ok
error in area
ok
error in area
ok
error in area
error in area
ok
error in area
error in area
ok
error in area
error in area
error in area
error in area
ok
error in area
error in area
error in area
error in area
error in area
error in area
ok
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in bath
ok
error in bed
error in bath
error in area
error in area
error in area
error in area
error in area
ok
error in bed
error in bath
error in bed
error in bath
error in area
error in bath
error in bed
error in bath
ok
error in area
error in area
error in bed
error in bath
error in bath
error in area
ok
error in area
error in bed
error in bath
error in bed
error in bath
error in area
ok
error in area
error in area
error in bed
error in bath
ok
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in area
error in area
error in area
ok
error in bed
error in bath
error in area
error in bed
error in bath
error in bath
error 

ok
error in area
error in bed
error in bath
error in bed
error in bath
error in area
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
ok
error in area
error in bed
error in bath
error in bed
error in bath
error in area
error in area
error in bed
error in bath
error in area
error in area
error in area
error in bed
error in bath
error in area
ok
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in bed
error in bath
error in area
error in bed
error in bath
error in bath
error in area
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
ok
error in area
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in area
error in bed
error in bath
error in bed
error in bath
error in area
error in bed
error in bath
error in bed
error in

## Scraping renting data - cork

In [32]:
import requests
from bs4 import BeautifulSoup
import csv
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


url = "https://www.daft.ie/property-for-rent/ireland"

# Mimicing a browser to avoid getting 403 error
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
pg_iter = np.arange(0,200, 20)

with open('Cork-extract-rent2.csv', 'w', encoding='utf-8', newline='') as f: 
    write_to_csv=csv.writer(f)
    
    # iterating through all the pages
    for page in pg_iter:
        s = url+"?from="+str(page)+"&location=cork&location=cork-city&location=cork-commuter-towns-cork"
        #Getting response from daft
        response = requests.get(s, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            print("ok")

            soup = BeautifulSoup(response.content, 'html.parser')
            each_ad = soup.find_all('li', class_="SearchPagestyled__Result-v8jvjf-2 iWPGnb")
            lc=0
            
           # pages = soup.find('p',attrs = {'data-testid':'pagination-results', 'class': 'SearchPagePagination__PaginationResults-sc-1gbvxa5-1 gzDAMl'})
           # print(pages, type(pages))
            
            for l in each_ad:
                address=''
                price=''
                bed=''
                bath=''
                area=''
                property_type=''
                ag=''
                agent=''
                lc=lc+1
              #  print(lc)
                
                try:
                    try:
                        address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 dzihyY").text             #address 

                    except:    
                       # print("Different format : need to scrape by clicking link")

                        try:
                            address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 hCMmam").text             #address 

                        except AttributeError: 
                            print("we have an error with address")
                            continue                  

                    # Find all a tags within the current li
                    all_links = l.find_all('a')

                   #print(lc)
                   # print(len(all_links))   

                    # Extract href attributes and print them

                    if len(all_links)==1:
                        for link in all_links:
                            new_url = "https://www.daft.ie/"+link.get('href')
                           # print(new_url)
                            soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')

                            try:                    
                                price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                            except:
                                print("error in price")

                            try:                    
                                bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                            except:
                                print("error in bed")

                            try:                    
                                bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                            except:
                                print("error in bath")

                            try:                    
                                area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                            except:
                                print("error in area")

                            try:                    
                                property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                            except:
                                print("error in type")


                    if len(all_links)>1:
                        a=0
                        for link in all_links:
                            if a>0:
                                new_url = "https://www.daft.ie/"+link.get('href')
                                #print(new_url)
                                soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')
                                try:                    
                                    price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                                except:
                                    print("error in price")

                                try:                    
                                    bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                                except:
                                    print("error in bed")
                                try:                    
                                    bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                                except:
                                    print("error in bath")

                                try:                    
                                    area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                                except:
                                    print("error in area")

                                try:                    
                                    property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                                except:
                                    print("error in type")

                            a=a+1

                    write_to_csv.writerow(["Cork",new_url, address, price, bed, bath, area, property_type])


                except AttributeError:
                    print("not sure why the error")
                
                

ok
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in bed
error in bath
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
ok
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
ok
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in bed
error in bath
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
ok
error in area
error in area
error in bed
error in bath
error in area
error in area
error in ar

## Scraping buying data - waterford

In [33]:
import requests
from bs4 import BeautifulSoup
import csv
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


url = "https://www.daft.ie/property-for-sale/ireland"

# Mimicing a browser to avoid getting 403 error
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
pg_iter = np.arange(0,340, 20)

with open('Waterford-extract-buy2.csv', 'w', encoding='utf-8', newline='') as f: 
    write_to_csv=csv.writer(f)
    
    # iterating through all the pages
    for page in pg_iter:
        s = url+"?from="+str(page)+"&location=waterford&location=waterford-city&location=waterford-city-centre-waterford"
        #Getting response from daft
        response = requests.get(s, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            print("ok")

            soup = BeautifulSoup(response.content, 'html.parser')
            each_ad = soup.find_all('li', class_="SearchPagestyled__Result-v8jvjf-2 iWPGnb")
            lc=0
            
           # pages = soup.find('p',attrs = {'data-testid':'pagination-results', 'class': 'SearchPagePagination__PaginationResults-sc-1gbvxa5-1 gzDAMl'})
           # print(pages, type(pages))
            
            for l in each_ad:
                address=''
                price=''
                bed=''
                bath=''
                area=''
                property_type=''
                ag=''
                agent=''
                lc=lc+1
              #  print(lc)
                
                try:
                    try:
                        address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 dzihyY").text             #address 

                    except:    
                       # print("Different format : need to scrape by clicking link")

                        try:
                            address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 hCMmam").text             #address 

                        except AttributeError: 
                            print("we have an error with address")
                            continue                  

                    # Find all a tags within the current li
                    all_links = l.find_all('a')

                   #print(lc)
                   # print(len(all_links))   

                    # Extract href attributes and print them

                    if len(all_links)==1:
                        for link in all_links:
                            new_url = "https://www.daft.ie/"+link.get('href')
                           # print(new_url)
                            soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')

                            try:                    
                                price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                            except:
                                print("error in price")

                            try:                    
                                bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                            except:
                                print("error in bed")

                            try:                    
                                bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                            except:
                                print("error in bath")

                            try:                    
                                area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                            except:
                                print("error in area")

                            try:                    
                                property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                            except:
                                print("error in type")


                    if len(all_links)>1:
                        a=0
                        for link in all_links:
                            if a>0:
                                new_url = "https://www.daft.ie/"+link.get('href')
                                #print(new_url)
                                soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')
                                try:                    
                                    price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                                except:
                                    print("error in price")

                                try:                    
                                    bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                                except:
                                    print("error in bed")
                                try:                    
                                    bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                                except:
                                    print("error in bath")

                                try:                    
                                    area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                                except:
                                    print("error in area")

                                try:                    
                                    property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                                except:
                                    print("error in type")

                            a=a+1

                    write_to_csv.writerow(["Waterford",new_url, address, price, bed, bath, area, property_type])


                except AttributeError:
                    print("not sure why the error")
                
                
    

ok
error in area
error in area
error in area
error in area
ok
error in area
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in bed
error in bath
ok
error in bath
error in area
error in bed
error in bath
error in area
error in area
ok
error in area
error in area
error in bed
error in bath
error in bed
error in bath
error in area
error in area
error in area
ok
error in bed
error in bath
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
ok
error in area
error in area
error in area
error in area
error in bed
error in bath
error in area
ok
error in bed
error in bath
error in area
error in area
error in area
error in area
error in area
ok
error in area
error in area
error in area
error in area
error in bath
error in area
error in area
error in area
ok
error in area
error in bed
error in bath
error in bed
error in bath
error in area
error in area
error in area
erro

## Scraping renting data - waterford


In [34]:
import requests
from bs4 import BeautifulSoup
import csv
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


url = "https://www.daft.ie/property-for-rent/ireland"

# Mimicing a browser to avoid getting 403 error
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
pg_iter = np.arange(0,40, 20)

with open('Waterford-extract-rent2.csv', 'w', encoding='utf-8', newline='') as f: 
    write_to_csv=csv.writer(f)
    
    # iterating through all the pages
    for page in pg_iter:
        s = url+"?from="+str(page)+"&location=waterford&location=waterford-city&location=waterford-city-centre-waterford"
        #Getting response from daft
        response = requests.get(s, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            print("ok")

            soup = BeautifulSoup(response.content, 'html.parser')
            each_ad = soup.find_all('li', class_="SearchPagestyled__Result-v8jvjf-2 iWPGnb")
            lc=0
            
           # pages = soup.find('p',attrs = {'data-testid':'pagination-results', 'class': 'SearchPagePagination__PaginationResults-sc-1gbvxa5-1 gzDAMl'})
           # print(pages, type(pages))
            
            for l in each_ad:
                address=''
                price=''
                bed=''
                bath=''
                area=''
                property_type=''
                ag=''
                agent=''
                lc=lc+1
              #  print(lc)
                
                try:
                    try:
                        address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 dzihyY").text             #address 

                    except:    
                       # print("Different format : need to scrape by clicking link")

                        try:
                            address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 hCMmam").text             #address 

                        except AttributeError: 
                            print("we have an error with address")
                            continue                  

                    # Find all a tags within the current li
                    all_links = l.find_all('a')

                   #print(lc)
                   # print(len(all_links))   

                    # Extract href attributes and print them

                    if len(all_links)==1:
                        for link in all_links:
                            new_url = "https://www.daft.ie/"+link.get('href')
                           # print(new_url)
                            soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')

                            try:                    
                                price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                            except:
                                print("error in price")

                            try:                    
                                bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                            except:
                                print("error in bed")

                            try:                    
                                bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                            except:
                                print("error in bath")

                            try:                    
                                area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                            except:
                                print("error in area")

                            try:                    
                                property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                            except:
                                print("error in type")


                    if len(all_links)>1:
                        a=0
                        for link in all_links:
                            if a>0:
                                new_url = "https://www.daft.ie/"+link.get('href')
                                #print(new_url)
                                soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')
                                try:                    
                                    price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                                except:
                                    print("error in price")

                                try:                    
                                    bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                                except:
                                    print("error in bed")
                                try:                    
                                    bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                                except:
                                    print("error in bath")

                                try:                    
                                    area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                                except:
                                    print("error in area")

                                try:                    
                                    property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                                except:
                                    print("error in type")

                            a=a+1

                    write_to_csv.writerow(["Waterford",new_url, address, price, bed, bath, area, property_type])


                except AttributeError:
                    print("not sure why the error")
                
                    

ok
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in bed
error in bath
error in area
error in area
error in area
error in area
ok
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area


In [25]:
s

'https://www.daft.ie/property-for-rent/ireland?from=20&location=waterford&location=waterford-city&location=waterford-city-centre-waterford'

## Scraping buying data - galway

In [35]:
import requests
from bs4 import BeautifulSoup
import csv
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


url = "https://www.daft.ie/property-for-sale/galway"

# Mimicing a browser to avoid getting 403 error
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
pg_iter = np.arange(0,1000, 20)

with open('Galway-extract-buy2.csv', 'w', encoding='utf-8', newline='') as f: 
    write_to_csv=csv.writer(f)
    
    # iterating through all the pages
    for page in pg_iter:
        s = url+"?from="+str(page)
        #Getting response from daft
        response = requests.get(s, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            print("ok")

            soup = BeautifulSoup(response.content, 'html.parser')
            each_ad = soup.find_all('li', class_="SearchPagestyled__Result-v8jvjf-2 iWPGnb")
            lc=0
            pages = soup.find('p',attrs = {'data-testid':'pagination-results', 'class': 'SearchPagePagination__PaginationResults-sc-1gbvxa5-1 gzDAMl'})
            print(pages, type(pages))
            for l in each_ad:
                address=''
                price=''
                bed=''
                bath=''
                area=''
                property_type=''
                ag=''
                agent=''
                lc=lc+1
              #  print(lc)
                try:
                    try:
                        address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 dzihyY").text             #address 

                    except:    
                       # print("Different format : need to scrape by clicking link")

                        try:
                            address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 hCMmam").text             #address 

                        except AttributeError: 
                            print("we have an error with address")
                            continue                  

                    # Find all <a> tags within the current <li> (l)
                    all_links = l.find_all('a')

                   #print(lc)
                   # print(len(all_links))   

                    # Extract href attributes and print them

                    if len(all_links)==1:
                        for link in all_links:
                            new_url = "https://www.daft.ie/"+link.get('href')
                           # print(new_url)
                            soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')

                            try:                    
                                price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                            except:
                                print("error in price")

                            try:                    
                                bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                            except:
                                print("error in bed")

                            try:                    
                                bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                            except:
                                print("error in bath")

                            try:                    
                                area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                            except:
                                print("error in area")

                            try:                    
                                property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                            except:
                                print("error in type")


                    if len(all_links)>1:
                        a=0
                        for link in all_links:
                            if a>0:
                                new_url = "https://www.daft.ie/"+link.get('href')
                                #print(new_url)
                                soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')
                                try:                    
                                    price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                                except:
                                    print("error in price")

                                try:                    
                                    bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                                except:
                                    print("error in bed")
                                try:                    
                                    bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                                except:
                                    print("error in bath")

                                try:                    
                                    area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                                except:
                                    print("error in area")

                                try:                    
                                    property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                                except:
                                    print("error in type")

                            a=a+1

                    write_to_csv.writerow(["Galway",new_url, address, price, bed, bath, area, property_type])


                except AttributeError:
                    print("not sure why the error")
                
                

ok
None <class 'NoneType'>
error in area
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in bed
error in bath
error in area
ok
None <class 'NoneType'>
error in area
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in area
ok
None <class 'NoneType'>
error in area
error in area
error in area
error in area
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
ok
None <class 'NoneType'>
error in bed
error in bath
error in bed
error in bath
error in bed
error in area
error in bed
error in area
error in area
error in bath
ok
None <class 'NoneType'>
error in area
error in area
error in area
error in area
error in area
ok
None <class 'NoneType'>
error in area
error in bed
error in bath
error in area
error in area
error in bath
error in area
error in bed
error in bath
error in bed
error in bath
error in area
error in bed
error in bath
err

error in bed
error in bath
error in bed
error in bath
error in area
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
ok
None <class 'NoneType'>
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in area
error in bath
error in area
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in bed
error in bath
error in area
ok
None <class 'NoneType'>
error in bed
error in bath
error in bed
error in bath
error in bed

## Scraping renting data - Galway

In [36]:
import requests
from bs4 import BeautifulSoup
import csv
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


url = "https://www.daft.ie/property-for-rent/dublin-city"

# Mimicing a browser to avoid getting 403 error
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
pg_iter = np.arange(0,100, 20)

with open('Galway-extract-rent2.csv', 'w', encoding='utf-8', newline='') as f: 
    write_to_csv=csv.writer(f)
    
    # iterating through all the pages
    for page in pg_iter:
        s = url+"?from="+str(page)+"&pageSize=0"
        #Getting response from daft
        response = requests.get(s, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            print("ok")

            soup = BeautifulSoup(response.content, 'html.parser')
            each_ad = soup.find_all('li', class_="SearchPagestyled__Result-v8jvjf-2 iWPGnb")
            lc=0
            
           # pages = soup.find('p',attrs = {'data-testid':'pagination-results', 'class': 'SearchPagePagination__PaginationResults-sc-1gbvxa5-1 gzDAMl'})
           # print(pages, type(pages))
            
            for l in each_ad:
                address=''
                price=''
                bed=''
                bath=''
                area=''
                property_type=''
                ag=''
                agent=''
                lc=lc+1
              #  print(lc)
                
                try:
                    try:
                        address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 dzihyY").text             #address 

                    except:    
                       # print("Different format : need to scrape by clicking link")

                        try:
                            address = l.find('h2', class_="TitleBlock__Address-sc-1avkvav-8 hCMmam").text             #address 

                        except AttributeError: 
                            print("we have an error with address")
                            continue                  

                    # Find all a tags within the current li
                    all_links = l.find_all('a')

                   #print(lc)
                   # print(len(all_links))   

                    # Extract href attributes and print them

                    if len(all_links)==1:
                        for link in all_links:
                            new_url = "https://www.daft.ie/"+link.get('href')
                           # print(new_url)
                            soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')

                            try:                    
                                price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                            except:
                                print("error in price")

                            try:                    
                                bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                            except:
                                print("error in bed")

                            try:                    
                                bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                            except:
                                print("error in bath")

                            try:                    
                                area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                            except:
                                print("error in area")

                            try:                    
                                property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                            except:
                                print("error in type")


                    if len(all_links)>1:
                        a=0
                        for link in all_links:
                            if a>0:
                                new_url = "https://www.daft.ie/"+link.get('href')
                                #print(new_url)
                                soup_2 = BeautifulSoup(requests.get(new_url, headers=headers).content, 'html.parser')
                                try:                    
                                    price = soup_2.find('h2', class_="TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq").text   #price 
                                except:
                                    print("error in price")

                                try:                    
                                    bed = soup_2.find('p', attrs = {'data-testid':'beds', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bedrooms
                                except:
                                    print("error in bed")
                                try:                    
                                    bath = soup_2.find('p', attrs = {'data-testid':'baths', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #No. of bathrooms
                                except:
                                    print("error in bath")

                                try:                    
                                    area = soup_2.find('p', attrs = {'data-testid':'floor-area', 'class': 'TitleBlock__CardInfoItem-sc-1avkvav-9 fgXVWJ'}).text  #area
                                except:
                                    print("error in area")

                                try:                    
                                    property_type = soup_2.find('p', attrs ={'data-testid':'property-type' , 'class':"TitleBlock__CardInfoItem-sc-1avkvav-9 cKZYAr"}).text   #type 
                                except:
                                    print("error in type")

                            a=a+1

                    write_to_csv.writerow(["Galway",new_url, address, price, bed, bath, area, property_type])


                except AttributeError:
                    print("not sure why the error")
                
                

ok
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in bed
error in bath
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in area
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in bed
error in bath
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in area
error in bed
error in bath
error in area
error in area
error in area
ok
error in bed
error in bath
error in area
error in area
error in area
error in bed
error in bath
error in area
error in 